<a href="https://colab.research.google.com/github/Shuvamjoy34/Multi-Class-Image-Classification-using-CNN-and-ANN/blob/main/VGG_16_of_CNN_car_vs_bike_vs_random_Image_Classification__with_default_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolutional Neural Network Based Car vs Bike vs Random Images Multi Class Classification**


**Importing required python packages**

In [ ]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.utils.data_utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.keras import balanced_batch_generator
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 

from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
from cv2 import cv2
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt

from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.utils import plot_model
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from numpy import array 
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

**Connecting google colab notebook to my google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing the image files with JPG format**


**Resizing and Labelling the images**




In [ ]:
data=[]
labels=[]
car=os.listdir("/content/drive/My Drive/Images/car/")
for cars in car:
    try:
        image=cv2.imread("/content/drive/My Drive/Images/car/"+ cars)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((224,224))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

bike=os.listdir("/content/drive/My Drive/Images/bike/")
for bikes in bike:
    try:
        image=cv2.imread("/content/drive/My Drive/Images/bike/"+ bikes)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((224,224))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")
random=os.listdir("/content/drive/My Drive/Images/random/")
for randoms in random:
    try:
        image=cv2.imread("/content/drive/My Drive/Images/random/"+ randoms)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((224,224))
        data.append(np.array(size_image))
        labels.append(2)
    except AttributeError:
        print("")

In [ ]:
images=np.array(data)
labels=np.array(labels)

In [ ]:
s=np.arange(images.shape[0])
np.random.shuffle(s)
images=images[s]
labels=labels[s]
num_classes=len(np.unique(labels))
len_data=len(images)

**Diving the image dataset into training, validation and testing dataset**

In [ ]:
images =images.astype(np.float32)
images = images/255

train_x , x , train_y , y = train_test_split(images , labels , 
                                            test_size = 0.2 ,
                                            random_state = 111)

eval_x , test_x , eval_y , test_y = train_test_split(x , y , 
                                                    test_size = 0.5 , 
                                                    random_state = 111)

plt.figure(1 , figsize = (15 ,5))
n = 0 
for z , j in zip([train_y , eval_y , test_y] , ['train labels','eval labels','test labels']):
    n += 1
    plt.subplot(1 , 3  , n)
    sns.countplot(x = z )
    plt.title(j)
plt.show()

In [ ]:
#x=x.reshape(x.shape[0], -1)
#y=y.reshape(y.shape[0], -1)

#train_x=train_x.reshape(train_x.shape[0], -1)
#train_y=train_y.reshape(train_y.shape[0], -1)
#eval_x=eval_x.reshape(eval_x.shape[0], -1)
#eval_y=eval_y.reshape(eval_y.shape[0], -1)


In [ ]:

from imblearn.over_sampling import SMOTE
#Over-sampling: SMOTE
#SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
#based on those that already exist. It works randomly picking a point from the minority class and computing 
#the k-nearest neighbors for this point.The synthetic points are added between the chosen point and its neighbors.
#We'll use ratio='minority' to resample the minority class.
#smote = SMOTE('minority',kind='regular',k_neighbors=2)

#train_x_sm, train_y_sm = smote.fit_sample(train_x,train_y)
print(train_x_sm.shape, train_y_sm.shape)

(107, 3072) (107,)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


**Encoding the labels of the target class of training,validation and testing dataset from(0,1,2) to (0,1) where 1 pops up when there is a class in the array else 0 pops up**

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)
eval_y = lb.transform(eval_y)


In [ ]:
from tensorflow.python.keras import regularizers
from keras.layers.normalization import BatchNormalization
#kernel_regularizer=regularizers.l2(0.05)


**Creating a basic Convolutional Neural Network architecture and using Random Search and Grid Search to find the best Hyper Parameters**

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
#vgg16
vggmodel = Sequential()
vggmodel.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
#keras.layers.BatchNormalization()
vggmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vggmodel.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
#keras.layers.BatchNormalization()
vggmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vggmodel.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
#keras.layers.BatchNormalization()
vggmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#keras.layers.BatchNormalization()
vggmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vggmodel.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
#keras.layers.BatchNormalization()
vggmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
keras.layers.Dropout(0.5)
vggmodel.add(Flatten())
vggmodel.add(Dense(units=4096,activation="relu"))
keras.layers.Dropout(0.5)
vggmodel.add(Dense(units=4096,activation="relu"))
keras.layers.Dropout(0.5)
vggmodel.add(Dense(units=3, activation="softmax"))

In [ ]:
vggmodel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 56, 56, 256)      

In [ ]:
from keras.optimizers import Adam
#opt = Adam(lr=1e-3, decay=1e-3 / 30)
vggmodel.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
#vggmodel.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
VGG=vggmodel.fit(train_x,train_y,batch_size = 32,epochs=100,verbose=1,validation_data=(eval_x, eval_y),callbacks=[checkpoint,early])
VGG

In [ ]:
#Once the model is trained we can evaluate it on Test data.

# Evaluating the model 
vggscore = vggmodel.evaluate(test_x, test_y, verbose=0)
print('Test Loss:', vggscore[0])
print('Test accuracy:', vggscore[1])

Test Loss: 1.0781618356704712
Test accuracy: 0.5


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = vggmodel.predict(test_x)
y_pred = np.argmax(Y_pred, axis=1)
target_names = ['class 0(car)', 'class 1(bike)','class 2(random)']
                                               
print(classification_report(np.argmax(test_y,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(test_y,axis=1), y_pred))

                 precision    recall  f1-score   support

   class 0(car)       0.00      0.00      0.00         4
  class 1(bike)       0.00      0.00      0.00         1
class 2(random)       0.50      1.00      0.67         5

       accuracy                           0.50        10
      macro avg       0.17      0.33      0.22        10
   weighted avg       0.25      0.50      0.33        10

[[0 0 4]
 [0 0 1]
 [0 0 5]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(vggmodel.history.history["accuracy"])
plt.plot(vggmodel.history.history['val_accuracy'])
plt.plot(vggmodel.history.history['loss'])
plt.plot(vggmodel.history.history['val_loss'])
plt.title("model accuracy & loss comparison")
plt.ylabel("Loss/Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()